# Test WANDB

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("mnm-shortlab-mit/allegro_hpo_vcrtiwzr")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv("project.csv")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/myless/.netrc
wandb: Currently logged in as: myless (mnm-shortlab-mit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
import pandas as pd
import ast
import numpy as np

# Read the CSV
df = pd.read_csv("project.csv")

# Function to safely extract metrics from the summary string
def extract_metrics(summary_str):
    try:
        summary_dict = ast.literal_eval(summary_str)
        runtime = summary_dict['_wandb']['runtime']
        val_rmse = summary_dict['validation_f_rmse']
        return runtime, val_rmse
    except:
        return None, None

# Extract metrics into new columns
df['runtime'], df['val_rmse'] = zip(*df['summary'].apply(extract_metrics))

# Remove rows with NaN values
df = df.dropna(subset=['runtime', 'val_rmse'])

# Convert runtime to hours
df['runtime_hours'] = df['runtime'] / 3600

# Function to find Pareto optimal points
def is_pareto_optimal(runtime, val_rmse):
    n_points = len(runtime)
    is_efficient = np.ones(n_points, dtype=bool)
    
    for i in range(n_points):
        for j in range(n_points):
            if i != j:
                if (runtime[j] <= runtime[i] and val_rmse[j] <= val_rmse[i] and 
                    (runtime[j] < runtime[i] or val_rmse[j] < val_rmse[i])):
                    is_efficient[i] = False
                    break
    
    return is_efficient

# Find Pareto optimal models
pareto_mask = is_pareto_optimal(df['runtime'].values, df['val_rmse'].values)
pareto_optimal = df[pareto_mask].sort_values('runtime')

# Display results
print("Found", len(pareto_optimal), "Pareto optimal models:")
for _, row in pareto_optimal.iterrows():
    print(f"\nModel: {row['name']}")
    print(f"Runtime: {row['runtime_hours']:.2f} hours")
    print(f"Validation RMSE: {row['val_rmse']:.4f}")

# Find best compromise model
# Normalize both metrics to 0-1 scale
runtime_norm = (pareto_optimal['runtime'] - pareto_optimal['runtime'].min()) / (pareto_optimal['runtime'].max() - pareto_optimal['runtime'].min())
rmse_norm = (pareto_optimal['val_rmse'] - pareto_optimal['val_rmse'].min()) / (pareto_optimal['val_rmse'].max() - pareto_optimal['val_rmse'].min())

# Calculate combined score (lower is better)
pareto_optimal['combined_score'] = runtime_norm + rmse_norm

# Get best compromise
best_compromise = pareto_optimal.loc[pareto_optimal['combined_score'].idxmin()]

print("\n\nBest Compromise Model:")
print(f"Model: {best_compromise['name']}")
print(f"Runtime: {best_compromise['runtime_hours']:.2f} hours")
print(f"Validation RMSE: {best_compromise['val_rmse']:.4f}")

# Get fastest and most accurate models for comparison
fastest = pareto_optimal.iloc[0]
most_accurate = pareto_optimal.loc[pareto_optimal['val_rmse'].idxmin()]

print("\nFastest Model:")
print(f"Model: {fastest['name']}")
print(f"Runtime: {fastest['runtime_hours']:.2f} hours")
print(f"Validation RMSE: {fastest['val_rmse']:.4f}")

print("\nMost Accurate Model:")
print(f"Model: {most_accurate['name']}")
print(f"Runtime: {most_accurate['runtime_hours']:.2f} hours")
print(f"Validation RMSE: {most_accurate['val_rmse']:.4f}")

Found 16 Pareto optimal models:

Model: allegro_model_0_r_max6p0_l_max0_num_layers1_num_tensor_features16_learning_rate0p0001
Runtime: 5.10 hours
Validation RMSE: 0.2842

Model: allegro_model_1_r_max6p0_l_max0_num_layers1_num_tensor_features16_learning_rate0p0005
Runtime: 5.19 hours
Validation RMSE: 0.2529

Model: allegro_model_2_r_max6p0_l_max0_num_layers1_num_tensor_features16_learning_rate0p001
Runtime: 5.24 hours
Validation RMSE: 0.2518

Model: allegro_model_4_r_max6p0_l_max0_num_layers1_num_tensor_features32_learning_rate0p0005
Runtime: 5.24 hours
Validation RMSE: 0.2480

Model: allegro_model_5_r_max6p0_l_max0_num_layers1_num_tensor_features32_learning_rate0p001
Runtime: 5.28 hours
Validation RMSE: 0.2440

Model: allegro_model_19_r_max6p0_l_max1_num_layers1_num_tensor_features16_learning_rate0p0005
Runtime: 5.30 hours
Validation RMSE: 0.2061

Model: allegro_model_20_r_max6p0_l_max1_num_layers1_num_tensor_features16_learning_rate0p001
Runtime: 5.35 hours
Validation RMSE: 0.2054

Mo

# Testing Elasticity

In [3]:
from forge.workflows.elasticity.lammps_elasticity import generate_lammps_elastic_input
from ase.io import read
from ase.build import bulk

#atoms = read('./scratch/data/mrs_neb_static/Cr4Ti7V111W4Zr2_14_to_15/00/POSCAR')
atoms = bulk('Fe', 'bcc', a=2.8665, cubic=True) * (4, 4, 4)
potential_file = '/Users/myless/Packages/mylammps/potentials/Fe_mm.eam.fs'
output_dir = './scratch/data/elasticity_test'
generate_lammps_elastic_input(atoms, potential_file, output_dir, job_name="elastic_calculation", use_gpu=True, use_kokkos=False)



[INFO] LAMMPS input files generated in: ./scratch/data/elasticity_test


# Database Working

In [9]:
from forge.core.database import DatabaseManager
from ase.io import read
import os

# 1. Your AWS database credentials
db_config = {
    'database': {
        'dbname': 'test_database',  # The name of the database you created on RDS
        'user': 'myless',           # Replace with your RDS username
        'password': 'vcrtiwzr',     # Replace with your RDS password
        'host': 'database-vcrtiwzr.cfg4i4qmuc4m.us-east-1.rds.amazonaws.com',
        'port': 5432                # Default Postgres port unless you changed it
    }
}

# 2. Instantiate the DatabaseManager
db_manager = DatabaseManager(config_dict=db_config)

# (Optional) If you need to drop and recreate tables, uncomment:
# with db_manager.conn.cursor() as cur:
#     cur.execute("""
#         DROP TABLE IF EXISTS calculations CASCADE;
#         DROP TABLE IF EXISTS structures CASCADE;
#     """)
# db_manager.conn.commit()
# db_manager._initialize_tables()

## Adding New Structures

In [10]:
from ase.io import read, write

data = read('./scratch/data/random_100_gen6.xyz', index=':')
#print(data[0])
#print(data[0].info)
#print(data[0].calc)
write('./scratch/data/random_100_gen6_fixed.xyz', data[5:])

In [12]:
db_manager.conn.rollback() # uncomment if your first add_structures_from_xyz fails

db_manager.add_structures_from_xyz(
    xyz_file='../data/gen_7-2025-02-12_og.xyz',
    skip_duplicates=True,
    default_model_type="vasp-static"
)

[INFO] Found 22201 frames in ../data/gen_7-2025-02-12_og.xyz
[INFO] Processing frame 0 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Skipping duplicate structure.
[INFO] Processing frame 1 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Added structure, ID=20197
[INFO] Added calculation, ID=20196
[INFO] Processing frame 2 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Added structure, ID=20198
[INFO] Added calculation, ID=20197
[INFO] Processing frame 3 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Added structure, ID=20199
[INFO] Added calculation, ID=20198
[INFO] Processing frame 4 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Added structure, ID=20200
[INFO] Added calculation, ID=20199
[INFO] Processing frame 5 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Added structure, ID=20201
[INFO] Added calculation, ID=20200
[INFO] Processing frame 6 from ../data/gen_7-2025-02-12_og.xyz...
[INFO] Added structure, ID=20202
[INFO] Added calculation, ID=20201
[INFO] Processing frame 7 from .

## Clearing Tables (Be Careful!)

In [2]:
# 0. Removing Duplicate Structures and their associated calculations
# Remove duplicates
duplicates = db_manager.remove_duplicate_structures(
    ltol=0.2,    # Length tolerance
    stol=0.3,    # Site tolerance
    angle_tol=5.0 # Angle tolerance in degrees
)

# Get a summary of what was removed
db_manager.get_duplicate_summary(duplicates)

Checking 162 structures for duplicates...
Found 1 duplicates of structure 1
Found 2 duplicates of structure 2
Found 1 duplicates of structure 3
Found 1 duplicates of structure 4
Found 1 duplicates of structure 5
Found 1 duplicates of structure 6
Found 1 duplicates of structure 7
Found 1 duplicates of structure 8
Found 6 duplicates of structure 9
Found 1 duplicates of structure 10
Found 1 duplicates of structure 11
Found 1 duplicates of structure 12
Found 1 duplicates of structure 13
Found 1 duplicates of structure 14
Found 1 duplicates of structure 15
Found 3 duplicates of structure 16
Found 2 duplicates of structure 17
Found 1 duplicates of structure 18
Found 1 duplicates of structure 19
Found 1 duplicates of structure 20
Found 1 duplicates of structure 21
Found 1 duplicates of structure 22
Found 2 duplicates of structure 23
Found 1 duplicates of structure 24
Found 1 duplicates of structure 25
Found 1 duplicates of structure 26
Found 1 duplicates of structure 27
Found 1 duplicates of 

In [4]:
# 1. Clear/Drop Tables
# -------------------
# BE CAREFUL: This will delete all data!
with db_manager.conn.cursor() as cur:
    cur.execute("""
        DROP TABLE IF EXISTS calculations CASCADE;
        DROP TABLE IF EXISTS structures CASCADE;
    """)
db_manager.conn.commit()
db_manager._initialize_tables()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


## Viewing Tables

In [2]:

# 2. View Tables
# -------------
# Count total structures and calculations
with db_manager.conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM structures")
    struct_count = cur.fetchone()[0]
    cur.execute("SELECT COUNT(*) FROM calculations")
    calc_count = cur.fetchone()[0]
print(f"Database contains {struct_count} structures and {calc_count} calculations")


Database contains 22087 structures and 22086 calculations


In [3]:

# View recent structures
with db_manager.conn.cursor() as cur:
    cur.execute("""
        SELECT structure_id, formula, composition, metadata->>'generation' as gen 
        FROM structures 
        ORDER BY structure_id DESC 
        LIMIT 5
    """)
    print("\nRecent structures:")
    for row in cur.fetchall():
        print(f"ID: {row[0]}, Formula: {row[1]}, Composition: {row[2]}, Generation: {row[3]}")




Recent structures:
ID: 22087, Formula: W252, Composition: {'W': {'at_frac': 1.0, 'num_atoms': 252}}, Generation: 6
ID: 22086, Formula: W252, Composition: {'W': {'at_frac': 1.0, 'num_atoms': 252}}, Generation: 6
ID: 22085, Formula: W252, Composition: {'W': {'at_frac': 1.0, 'num_atoms': 252}}, Generation: 6
ID: 22084, Formula: W252, Composition: {'W': {'at_frac': 1.0, 'num_atoms': 252}}, Generation: 6
ID: 22083, Formula: W53, Composition: {'W': {'at_frac': 1.0, 'num_atoms': 53}}, Generation: 6


In [4]:
# View calculations for a specific structure
db_manager.get_calculations(20191)

[{'id': 20191,
  'model_type': 'vasp-static',
  'energy': [-1034.10503071],
  'forces': array([[-9.229690e-01, -4.506000e-02,  1.344966e+00],
         [ 6.280400e-01,  2.901000e-03,  1.308524e+00],
         [-4.486000e-03,  1.293928e+00,  1.747160e-01],
         [-2.559973e+00,  1.277421e+00,  1.209620e-01],
         [ 6.437550e-01, -6.355900e-02, -5.342630e-01],
         [ 8.895380e-01, -3.517149e+00, -7.620430e-01],
         [-5.480000e-03,  4.239460e-01, -5.936040e-01],
         [ 1.559373e+00, -7.585400e-02, -9.632360e-01],
         [ 1.239175e+00, -7.066690e-01,  8.629400e-02],
         [-1.607800e-02,  8.207010e-01,  7.601330e-01],
         [ 1.130363e+00, -6.038600e-02, -8.498770e-01],
         [ 4.934040e-01,  2.622390e-01,  7.058600e-02],
         [ 8.845900e-02, -2.536150e-01,  1.254863e+00],
         [ 3.617710e-01,  3.201730e-01, -2.382980e-01],
         [-1.630334e+00, -6.509110e-01,  1.855767e+00],
         [-3.037850e-01, -2.719130e-01, -5.943230e-01],
         [-5.97950

## Search Examples

In [17]:
# 3. Search Examples
# ----------------
# Find structures with specific composition with debugging enabled
cr_structures = db_manager.find_structures(
    elements=['Cr'],  # Must contain Cr
    composition_constraints={
        'Cr': (0.1, 1.0)  # At least 1% Cr
    },
    debug=False  # Enable debugging
)

print(db_manager.get_structure(cr_structures[4]))
print(db_manager.get_calculations(cr_structures[4]))

Atoms(symbols='Cr25Ti25V24W25Zr25', pbc=True, cell=[[12.552320703, 0.033758657, -4.406501718], [-6.249853009, 10.8645436, -4.414950182], [0.001908008, -0.0071473, 13.224785071]])
[{'id': 16, 'model_type': 'vasp-static', 'energy': [-1113.77488505], 'forces': array([[-1.329357, -0.645809,  1.809289],
       [-0.191765, -2.395044,  0.301439],
       [ 0.405569,  0.890876,  5.05648 ],
       [ 0.03796 , -1.18591 , -0.895091],
       [ 0.169056,  2.336213,  0.369428],
       [ 1.167045,  0.076883,  0.661512],
       [-0.79992 ,  0.853109, -0.254597],
       [ 3.393114, -0.264754, -0.951341],
       [ 0.703927, -0.063254,  1.029836],
       [-1.047128,  0.833863, -2.079239],
       [ 0.336355, -0.739547,  0.062264],
       [ 2.439446,  1.178822,  1.689913],
       [ 0.405759,  1.055313, -0.808278],
       [ 1.721694,  0.997199,  0.689911],
       [-1.180354, -2.334016,  1.600422],
       [-1.089908,  2.214927, -0.593671],
       [ 0.91121 , -2.152509,  1.148381],
       [ 1.743501,  1.21594 

In [5]:
# Find structures with no calculations at all
no_calcs = db_manager.find_structures_without_calculation()

# Find structures without completed VASP calculations
no_vasp = db_manager.find_structures_without_calculation(model_type="vasp*", status="completed")

# Find structures without any VASP calculations (regardless of status)
no_vasp_any_status = db_manager.find_structures_without_calculation(model_type="vasp*", status=None)

print(f"Found {len(no_calcs)} structures with no calculations at all")
print(f"Found {len(no_vasp)} structures without completed VASP calculations")
print(f"Found {len(no_vasp_any_status)} structures without any VASP calculations (regardless of status)")

print(no_calcs)
print(db_manager.get_structure(no_calcs[0]))

Found 1 structures with no calculations at all
Found 1 structures without completed VASP calculations
Found 1 structures without any VASP calculations (regardless of status)
[20196]
Atoms(symbols='Cr117', pbc=True, cell=[8.535340639, 11.380454186, 14.225567732])


In [2]:
atoms = db_manager.get_atoms_with_calculation(structure_id=11245, model_type="vasp-static")
print(atoms)
print(atoms.info)

from ase.io import write
write('../data/test_atoms.xyz', atoms)


Atoms(symbols='Cr14Ti18V74W8Zr10', pbc=True, cell=[[12.612306025, -0.036812695, -4.453065183], [-6.303661977, 10.90479068, -4.356323681], [-0.012379756, 0.097350379, 13.323985214]], forces=...)
{'batch_id': True, 'generation': 0, 'composition': True, 'config_type': True, 'source_path': 'V-Cr-Ti-W-Zr/Fixed_Engrads/vcrtiwzr_fin_engrads/job_gen_0-2024-11-01/neb/structure_8238', 'temperature': True, 'date_processed': '2025-01-03T17:45:52.834447', 'structure_type': 'neb', 'structure_index': 8238, 'adversarial_step': True, 'energy': -1116.40199072, 'stress': array([ 1.20887359e-02,  6.27784095e-03,  8.11046662e-03,  1.12689199e-03,
       -7.21143964e-04,  5.04563598e-05]), 'calculation_id': 11245, 'model_type': 'vasp-static'}


In [13]:
# Find structures with high forces
high_force_structures = db_manager.find_structures(
    calculation_constraints={
        'model_type': 'vasp-static',
        'forces_max_magnitude': 0.5  # eV/Å
    }
)
print(f"\nFound {len(high_force_structures)} structures with forces >0.5 eV/Å")



TypeError: DatabaseManager.find_structures() got an unexpected keyword argument 'calculation_constraints'

In [14]:
# Find structures from a specific generation
gen5_structures = db_manager.find_structures(
    metadata_constraints={
        'generation': 5
    }
)
print(f"\nFound {len(gen5_structures)} structures from generation 5")

TypeError: DatabaseManager.find_structures() got an unexpected keyword argument 'metadata_constraints'

In [3]:
# Combine multiple constraints
specific_structures = db_manager.find_structures(
    elements=['V', 'Cr'],
    composition_constraints={
        'V': (0.4, 0.6),    # 40-60% V
        'Cr': (0.1, 0.2)    # 10-20% Cr
    },
    metadata_constraints={
        'generation': 5,
        'structure_type': 'vac'
    },
    calculation_constraints={
        'model_type': 'vasp-static',
        'energy': (-1200, -1000)  # Energy range in eV
    }
)
print(f"\nFound {len(specific_structures)} structures matching all criteria")

TypeError: DatabaseManager.find_structures() got an unexpected keyword argument 'metadata_constraints'

## Examine Structure Details

In [2]:

# 4. Examine Structure Details
# --------------------------
# Get full details for a specific structure
def examine_structure(db_manager, structure_id):
    """Get detailed information about a structure and its calculations."""
    with db_manager.conn.cursor() as cur:
        # Get structure details
        cur.execute("""
            SELECT formula, composition, metadata
            FROM structures
            WHERE structure_id = %s
        """, (structure_id,))
        row = cur.fetchone()
        if row:
            print(f"\nStructure {structure_id}:")
            print(f"Formula: {row[0]}")
            print(f"Composition: {row[1]}")
            print(f"Metadata: {row[2]}")
            
            # Get associated calculations
            cur.execute("""
                SELECT model_type, energy, stress, metadata
                FROM calculations
                WHERE structure_id = %s
            """, (structure_id,))
            print("\nCalculations:")
            for calc in cur.fetchall():
                print(f"- {calc[0]}: E={calc[1]:.3f} eV")
                if calc[2]:  # stress
                    print(f"  Stress tensor available")
                if calc[3]:  # metadata
                    status = calc[3].get('status', 'unknown')
                    print(f"  Status: {status}")
        else:
            print(f"No structure found with ID {structure_id}")

# Usage example:
if specific_structures:
    examine_structure(db_manager, specific_structures[0])

NameError: name 'specific_structures' is not defined

In [7]:
random_structure = db_manager.get_structure_metadata(11245)
print(random_structure['config_type'])


True


# Fixing the config_type for Database

In [141]:
from forge.core.database import DatabaseManager
from ase.io import read
import os

# 1. Your AWS database credentials
db_config = {
    'database': {
        'dbname': 'test_database',  # The name of the database you created on RDS
        'user': 'myless',           # Replace with your RDS username
        'password': 'vcrtiwzr',     # Replace with your RDS password
        'host': 'database-vcrtiwzr.cfg4i4qmuc4m.us-east-1.rds.amazonaws.com',
        'port': 5432                # Default Postgres port unless you changed it
    }
}

# 2. Instantiate the DatabaseManager
db_manager = DatabaseManager(config_dict=db_config)

In [142]:
with db_manager.conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM structures")
    struct_count = cur.fetchone()[0]
    cur.execute("SELECT COUNT(*) FROM calculations")
    calc_count = cur.fetchone()[0]
print(f"Database contains {struct_count} structures and {calc_count} calculations")

Database contains 22090 structures and 22086 calculations


In [95]:
# Find structures with no calculations at all
no_calcs = db_manager.find_structures_without_calculation()

# Find structures without completed VASP calculations
no_vasp = db_manager.find_structures_without_calculation(model_type="vasp*", status="completed")

# Find structures without any VASP calculations (regardless of status)
no_vasp_any_status = db_manager.find_structures_without_calculation(model_type="vasp*", status=None)

print(f"Found {len(no_calcs)} structures with no calculations at all")
print(f"Found {len(no_vasp)} structures without completed VASP calculations")
print(f"Found {len(no_vasp_any_status)} structures without any VASP calculations (regardless of status)")

Found 4 structures with no calculations at all
Found 4 structures without completed VASP calculations
Found 4 structures without any VASP calculations (regardless of status)


In [13]:
print(no_calcs)

[20196, 22088, 22089, 22090]


In [14]:

for i in range(len(no_calcs)):
    print(db_manager.get_structure(no_calcs[i]))
    print(db_manager.get_structure_metadata(no_calcs[i]))


Atoms(symbols='Cr117', pbc=True, cell=[8.535340639, 11.380454186, 14.225567732])
{'REF_energy': -1094.8750554, 'REF_stress': [-0.0015125236310026676, -0.00030062228704816715, -0.001466842025710328, -0.00030062228704816715, -0.00461909748521029, 0.0014221590618781299, -0.001466842025710328, 0.0014221590618781299, -0.0019115994830025175], 'generation': 7, 'source_path': 'V-Cr-Ti-W-Zr/Fixed_Engrads/vcrtiwzr_fin_engrads/job_gen_7-2025-02-12/Cr/Cr117_tri-vacancy_idx_3427', 'date_processed': '2025-02-12T16:43:40.980844'}
Atoms(symbols='Cr117', pbc=True, cell=[8.535340639, 11.380454186, 14.225567732])
{'loss': 0.009863905608654022, 'final': True, 'energy': -1094.5919189453125, 'variance': 0.00986368115991354, 'parent_id': 20196, 'REF_energy': -1094.8750554, 'REF_stress': [-0.0015125236310026676, -0.00030062228704816715, -0.001466842025710328, -0.00030062228704816715, -0.00461909748521029, 0.0014221590618781299, -0.001466842025710328, 0.0014221590618781299, -0.0019115994830025175], 'generation

In [15]:
db_manager.get_structure(11245)


Atoms(symbols='Cr14Ti18V74W8Zr10', pbc=True, cell=[[12.612306025, -0.036812695, -4.453065183], [-6.303661977, 10.90479068, -4.356323681], [-0.012379756, 0.097350379, 13.323985214]])

In [33]:
def clean_and_update_structure_metadata(db_manager, structure_ids=None, dry_run=False):
    """
    Process structure metadata to:
    1. Remove calculation-related keys
    2. Set config_type based on source_path patterns
    
    Args:
        db_manager: DatabaseManager instance
        structure_ids: List of structure IDs to process (if None, process all)
        dry_run: If True, print changes but don't apply them
        
    Returns:
        Dict with statistics about the changes made
    """
    # Keys to remove from metadata
    keys_to_remove = ['forces', 'REF_energy', 'energy', 'REF_stress', 'stress']
    
    # Get all structure IDs if not provided
    if structure_ids is None:
        with db_manager.conn.cursor() as cur:
            cur.execute("SELECT structure_id FROM structures ORDER BY structure_id")
            structure_ids = [row[0] for row in cur.fetchall()]
    
    # Statistics
    stats = {
        'total': len(structure_ids),
        'processed': 0,
        'skipped_no_source_path': 0,
        'config_types': {}
    }
    
    # Process each structure
    for i, struct_id in enumerate(structure_ids):
        # Print progress every 1000 structures
        if i % 1000 == 0:
            print(f"Processing structure {i}/{len(structure_ids)}...")
        
        try:
            # Get current metadata
            metadata = db_manager.get_structure_metadata(struct_id)
            if metadata is None:
                print(f"Warning: No metadata found for structure {struct_id}")
                continue
                
            # Make a copy to modify
            new_metadata = metadata.copy()
            
            # Remove unwanted keys
            for key in keys_to_remove:
                if key in new_metadata:
                    new_metadata.pop(key)
            
            # Check for source_path
            if 'source_path' not in new_metadata:
                stats['skipped_no_source_path'] += 1
                continue
                
            source_path = new_metadata['source_path']
            
            # Determine config_type based on source_path
            config_type = None
            
            # Check for adversarial first (since it can be combined with others)
            is_adversarial = 'adversarial' in source_path
            
            # Check for primary types
            if 'neb' in source_path:
                config_type = 'neb'
            elif 'vac' in source_path:
                config_type = 'vac'
            elif 'perf' in source_path:
                config_type = 'perf'
            elif 'comp' in source_path and 'temp' in source_path:
                config_type = 'comp-explore'
            else:
                # If no recognized pattern, keep existing or set to 'unknown'
                config_type = new_metadata.get('config_type', 'unknown')
            
            # Add adversarial suffix if needed
            if is_adversarial and config_type != 'unknown':
                config_type = f"{config_type}_aa-mc"
            
            # Update the config_type
            new_metadata['config_type'] = config_type
            
            # Update statistics
            if config_type not in stats['config_types']:
                stats['config_types'][config_type] = 0
            stats['config_types'][config_type] += 1
            
            # Update the metadata if changes were made
            if new_metadata != metadata:
                if not dry_run:
                    db_manager.update_structure_metadata(struct_id, new_metadata)
                else:
                    print(f"Would update structure {struct_id}: {metadata.get('config_type', 'None')} -> {config_type}")
            
            stats['processed'] += 1
            
        except Exception as e:
            print(f"Error processing structure {struct_id}: {e}")
    
    # Print summary
    print("\nProcessing complete!")
    print(f"Total structures: {stats['total']}")
    print(f"Successfully processed: {stats['processed']}")
    print(f"Skipped (no source_path): {stats['skipped_no_source_path']}")
    print("\nConfig type distribution:")
    for config_type, count in sorted(stats['config_types'].items()):
        print(f"  {config_type}: {count}")
    
    return stats

In [37]:
import numpy as np
random_structures = np.random.randint(0, struct_count, 500).tolist()
print(random_structures)

[7843, 19720, 13981, 14627, 11111, 2414, 21787, 9297, 18018, 14776, 12792, 17742, 9267, 4530, 11852, 3140, 12088, 7700, 14246, 16989, 16656, 11110, 124, 5631, 8293, 6861, 13916, 4244, 18452, 2452, 7495, 3733, 19451, 7047, 2634, 5294, 8966, 16193, 10529, 5127, 8482, 12681, 13705, 14237, 20843, 14636, 4599, 21165, 10282, 5226, 4554, 8763, 3854, 11493, 216, 1688, 8266, 10433, 5646, 6303, 20395, 6253, 16794, 18965, 21403, 7052, 11532, 16978, 11636, 16286, 5571, 5879, 19845, 6985, 15497, 14682, 19747, 15449, 5565, 8634, 7465, 21456, 11813, 3316, 14606, 14842, 9296, 10775, 15278, 16644, 2291, 10547, 2875, 13195, 6660, 13166, 17688, 20788, 13581, 3921, 11165, 13567, 167, 13366, 18290, 8776, 16394, 9779, 13151, 14811, 16606, 3509, 4112, 18944, 9221, 671, 18618, 4956, 11187, 1880, 5412, 21587, 18585, 18558, 11956, 22045, 5227, 17894, 11120, 15420, 5944, 21188, 16767, 4625, 15615, 16488, 20297, 14392, 12307, 4136, 1419, 14398, 8837, 5590, 10167, 1950, 2967, 9181, 3565, 19460, 13217, 21703, 3403,

In [94]:
clean_and_update_structure_metadata(db_manager=db_manager, dry_run=True)

Processing structure 0/22090...
Processing structure 1000/22090...
Processing structure 2000/22090...
Processing structure 3000/22090...
Processing structure 4000/22090...
Processing structure 5000/22090...
Processing structure 6000/22090...
Processing structure 7000/22090...
Processing structure 8000/22090...
Processing structure 9000/22090...
Processing structure 10000/22090...
Processing structure 11000/22090...
Processing structure 12000/22090...
Processing structure 13000/22090...
Processing structure 14000/22090...
Processing structure 15000/22090...
Processing structure 16000/22090...
Processing structure 17000/22090...
Processing structure 18000/22090...
Processing structure 19000/22090...
Processing structure 20000/22090...
Processing structure 21000/22090...
Processing structure 22000/22090...

Processing complete!
Total structures: 22090
Successfully processed: 22090
Skipped (no source_path): 0

Config type distribution:
  A15: 252
  C15: 260
  bcc_distorted: 146
  comp-expl

{'total': 22090,
 'processed': 22090,
 'skipped_no_source_path': 0,
 'config_types': {'vac_aa-mc': 2444,
  'neb': 8017,
  'comp-explore': 3609,
  'neb_aa-mc': 3486,
  'dia': 253,
  'vac': 2238,
  'liquid': 109,
  'sia': 76,
  'di-sia': 14,
  'surf_liquid': 48,
  'surface_100': 108,
  'surface_110': 117,
  'surface_111': 114,
  'surface_112': 129,
  'dimer': 41,
  'hcp': 168,
  'short_range': 250,
  'phonon': 72,
  'C15': 260,
  'A15': 252,
  'unknown': 19,
  'gamma_surface': 120,
  'bcc_distorted': 146}}

In [98]:
def find_structures_with_config_type_true(db_manager):
    """
    Find all structures where metadata contains 'config_type' = True
    
    Args:
        db_manager: DatabaseManager instance
        
    Returns:
        List of structure IDs
    """
    query = """
    SELECT structure_id 
    FROM structures 
    WHERE metadata->>'config_type' = 'Unknown' OR metadata->>'config_type' = 'unknown'
    ORDER BY structure_id
    """
    
    structure_ids = []
    try:
        with db_manager.conn.cursor() as cur:
            cur.execute(query)
            structure_ids = [row[0] for row in cur.fetchall()]
            
        print(f"Found {len(structure_ids)} structures with 'config_type' = unknown")
        if structure_ids:
            print(f"First few IDs: {structure_ids[:5]}")
            
        return structure_ids
        
    except Exception as e:
        print(f"Error executing query: {e}")
        return []

In [97]:
# Find structures with config_type = True
problematic_ids = find_structures_with_config_type_true(db_manager)

# Optionally, examine one of these structures to see its full metadata
if problematic_ids:
    sample_id = problematic_ids[0]
    metadata = db_manager.get_structure_metadata(sample_id)
    print(f"Sample metadata for structure {sample_id}:")
    print(metadata)

Found 19 structures with 'config_type' = unknown
First few IDs: [20211, 20212, 20213, 20721, 20722]
Sample metadata for structure 20211:
{'generation': 7, 'config_type': 'unknown', 'source_path': 'V-Cr-Ti-W-Zr/Fixed_Engrads/vcrtiwzr_fin_engrads/job_gen_7-2025-02-12/Cr/Cr122_di-sia_idx_3476', 'date_processed': '2025-02-12T16:43:42.028469'}


In [139]:
N = 18
og_metadata = db_manager.get_structure_metadata(problematic_ids[N])
print('config_type: ', og_metadata['config_type'])
print('source_path: ', og_metadata['source_path'])

config_type:  di-sia
source_path:  V-Cr-Ti-W-Zr/Fixed_Engrads/vcrtiwzr_fin_engrads/job_gen_6-2024-12-23/W_Big/W252_di-SIA_idx_3648


In [137]:
og_metadata['config_type'] = 'di-sia'
db_manager.update_structure_metadata(problematic_ids[N], og_metadata)
print('config_type: ', db_manager.get_structure_metadata(problematic_ids[N])['config_type'])

config_type:  di-sia


In [146]:
print(db_manager.get_calculations(124)[0].keys())

dict_keys(['id', 'model_type', 'energy', 'forces', 'stress', 'variance', 'error', 'status', 'gpu_count', 'parameters', 'date_started', 'gpu_memory_mb', 'model_version', 'date_completed', 'runtime_seconds'])


In [147]:
db_manager.get_structure_metadata(124)

{'batch_id': 11,
 'generation': 2,
 'composition': True,
 'config_type': 'neb_aa-mc',
 'source_path': 'V-Cr-Ti-W-Zr/Fixed_Engrads/vcrtiwzr_fin_engrads/job_gen_2-2024-11-10_selected_fin/batch_11/structure_4_neb_37_adversarial_2649',
 'temperature': True,
 'date_processed': '2025-01-03T17:27:25.183957',
 'structure_type': 'neb',
 'structure_index': 4,
 'adversarial_step': 2649}

In [153]:
def update_vac_to_vacancy_config_type(db_manager, initial_config_type, final_config_type, dry_run=False):
    """
    Find all structures where metadata contains 'config_type' = 'vac'
    and update them to 'config_type' = 'vacancy'
    
    Args:
        db_manager: DatabaseManager instance
        dry_run: If True, only print what would be changed without making changes
        
    Returns:
        List of updated structure IDs
    """
    query = """
    SELECT structure_id 
    FROM structures 
    WHERE metadata->>'config_type' = '{initial_config_type}'
    ORDER BY structure_id
    """
    
    structure_ids = []
    updated_count = 0
    
    try:
        with db_manager.conn.cursor() as cur:
            cur.execute(query)
            structure_ids = [row[0] for row in cur.fetchall()]
            
        print(f"Found {len(structure_ids)} structures with 'config_type' = '{initial_config_type}'")
        if structure_ids:
            print(f"First few IDs: {structure_ids[:5]}")
        
        # Update each structure's metadata
        for struct_id in structure_ids:
            try:
                # Get current metadata
                metadata = db_manager.get_structure_metadata(struct_id)
                
                # Make a copy and update config_type
                new_metadata = metadata.copy()
                new_metadata['config_type'] = final_config_type
                
                # Update the metadata
                if not dry_run:
                    db_manager.update_structure_metadata(struct_id, new_metadata)
                    updated_count += 1
                else:
                    print(f"Would update structure {struct_id}: config_type '{initial_config_type}' -> '{final_config_type}'")
            except Exception as e:
                print(f"Error updating structure {struct_id}: {e}")
        
        print(f"Updated {updated_count} structures" if not dry_run else "Dry run complete - no changes made")
        return structure_ids
        
    except Exception as e:
        print(f"Error executing query: {e}")
        return []

In [156]:
update_vac_to_vacancy_config_type(db_manager,
                                  initial_config_type='vacancy', 
                                  final_config_type='vacancy-alloy_aa-mc', 
                                  dry_run=True)

Found 0 structures with 'config_type' = 'vacancy'
Dry run complete - no changes made


[]